# Consdb for LSSTComCam for visits

- Creation date : 2025-04-18
- last update : 2025-04-19
- https://usdf-rsp-dev.slac.stanford.edu/consdb/
- Schemes for constdb : https://sdm-schemas.lsst.io/
- Documentation : https://consdb.lsst.io/index.html


LSSTCam/runs/nightlyValidation/{day_obs}/<lsst_distrib_tag>/DM-50157

In [ ]:
from lsst.summit.utils import ConsDbClient

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import matplotlib as mpl
import matplotlib.cm as cm 
import matplotlib.colors as colors
import matplotlib.cm as cmx
from matplotlib.colors import ListedColormap

import seaborn as sns
%matplotlib widget

from lsst.meas.algorithms.installGaussianPsf import FwhmPerSigma

from tqdm.notebook import tqdm
import pandas as pd

In [ ]:
#xx-small
#x-small
#small
#medium
#large
#x-large
#xx-large

plt.rcParams["figure.figsize"] = (10,6)
plt.rcParams["axes.labelsize"] = 'x-large'
plt.rcParams['axes.titlesize'] = 'x-large'
plt.rcParams['xtick.labelsize']= 'x-large'
plt.rcParams['ytick.labelsize']= 'x-large'

In [ ]:
from astropy.table import Table, join
from astropy.time import Time

from astropy.coordinates import SkyCoord
from astropy import coordinates
import astropy.coordinates as coord
import astropy.units as u

In [ ]:
import lsst.geom as geom
import lsst
from lsst.geom import Angle 

In [ ]:
# https://pipelines.lsst.io/modules/lsst.geom/getting-started.html
func_degToRad = lambda x : Angle(x,lsst.geom.degrees).asRadians()
func_zendtoAirmass = lambda x : 1./np.cos(func_degToRad(x))
#func_wrap = lambda x : Angle(x,lsst.geom.radians).wrap(180.*lsst.geom.degrees)

## Configuration

In [ ]:
instrument = "LSSTComCam"

In [ ]:
os.environ["no_proxy"] += ",.consdb"

In [ ]:
url="http://consdb-pq.consdb:8080/consdb"

In [ ]:
consdb=ConsDbClient(url)

https://sdm-schemas.lsst.io/cdb_lsstcomcam.html#exposure

In [ ]:
# Query both consDB tables
#exposure = consdb.query("SELECT * FROM cdb_lsstcam.exposure WHERE science_program = 'BLOCK-351'")
#visits = consdb.query("SELECT * FROM cdb_lsstcam.visit1 WHERE science_program = 'BLOCK-351'")
#visits_ql = consdb.query("SELECT * FROM cdb_lsstcam.visit1_quicklook")
#visits_ql = consdb.query("SELECT * FROM cdb_lsstcam.visit1")

exposure = consdb.query("SELECT * FROM cdb_lsstcomcam.exposure WHERE day_obs >= 20241021")
visits = consdb.query("SELECT * FROM cdb_lsstcomcam.visit1 WHERE day_obs >= 20241021")
#visits_ql = consdb.query("SELECT * FROM cdb_lsstcam.visit1_quicklook")
visits_ql = consdb.query("SELECT * FROM cdb_lsstcomcam.visit1 WHERE day_obs >= 20241021")

# Join using astropy's join function on 'visit_id'
exposure_join = exposure.rename_column("exposure_id", "visit_id")
merged_exposure = join(exposure, visits, keys="visit_id", join_type="inner")  
merged_visits = join(visits, visits_ql, keys="visit_id", join_type="inner")  

# Display or use the merged table
#print(merged_visits)

## Explore the visit info in consdb

In [ ]:
print(visits.columns)

In [ ]:
df_visits = visits.to_pandas()

In [ ]:
df_visits['band'].unique()

### Observation dates

In [ ]:
df_visits["day_obs"].unique()

### Filters

In [ ]:
print(df_visits["physical_filter"].unique(),df_visits["band"].unique())

In [ ]:
df_visits = df_visits[df_visits.band != 'other']
df_visits = df_visits[df_visits.band != 'none']

## Reasonable airmass

In [ ]:
df_visits = df_visits[df_visits["airmass"]>=1.0]

### Science program and observation reason

In [ ]:
df_visits["science_program"].unique()

In [ ]:
df_visits['observation_reason'].unique()

### Time exposure

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,4))
ax.hist(df_visits["exp_time"].values,bins=50,facecolor="b");
plt.show()

## Region in sky

In [ ]:
dict_b_to_col = {"u": "b", "g":"g", "r": "r", "i": 'orange', 'z': "magenta",'y':'purple','other':"k",'none':'grey'}

In [ ]:
col = df_visits["band"].map(lambda b:  dict_b_to_col[b] ).values

In [ ]:
col

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,4))
df_visits.plot.scatter(x='s_ra',y='s_dec',ax=ax,c=col)
ax.grid()
ax.set_aspect("equal")
plt.show()

In [ ]:
palette_spectral = sns.color_palette("Spectral_r", as_cmap=True)

In [ ]:
cmap_time = ListedColormap(sns.color_palette("Spectral_r", df_visits.size))

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(10,4))
ax1,ax2 = axs.flatten() 
df_visits['s_ra'].hist(bins=50,ax=ax1)
df_visits['s_dec'].hist(bins=50,ax=ax2)

In [ ]:
df_coordinates = df_visits[['s_ra','s_dec','obs_start_mjd','airmass']]

In [ ]:
df_coordinates 

## Must remove not a number in ra and dec

In [ ]:
df_dtypes = np.array(df_coordinates.dtypes)
df_numericDtypes= [x.kind in 'bifc' for x in df_dtypes]

In [ ]:
df_numericDtypes

In [ ]:
from pandas.api.types import is_any_real_numeric_dtype

In [ ]:
is_any_real_numeric_dtype(df_coordinates ['s_ra'])

In [ ]:
is_any_real_numeric_dtype(df_coordinates ['s_dec'])

In [ ]:
is_any_real_numeric_dtype(df_coordinates ['obs_start_mjd'])

In [ ]:
df_ = df_coordinates .copy().apply(pd.to_numeric, errors='coerce')
test_nmr_coerce = df_.dropna(axis=0, how='any')
display(test_nmr_coerce)

In [ ]:
df_coordinates  = test_nmr_coerce

In [ ]:
ra_g = df_coordinates['s_ra'].apply(func_degToRad) 
dec_g = df_coordinates['s_dec'].apply(func_degToRad) 

In [ ]:
dt = df_coordinates['obs_start_mjd'].values - df_coordinates['obs_start_mjd'].min()

In [ ]:
dtmin = dt.min()
dtmax = dt.max()

In [ ]:
# Galactic plane
gal_long = np.linspace(-180.,180,360)
gal_lat = np.zeros((360))
coordinates_galactic_planes = SkyCoord(l=gal_long*u.degree, b=gal_lat*u.degree, frame='galactic')
gp_radec = coordinates_galactic_planes.transform_to('icrs')  
# for galactic plane
# gp_radec.ra : 0., 360.
# gp_radec.dec : -90, 90
gp_ra_toplot = coordinates.Angle(gp_radec.ra.degree*u.degree)
gp_ra_toplot = gp_ra_toplot.wrap_at(180*u.degree)
# gp_ra_toplot -180, 180

In [ ]:
ra = coordinates.Angle(df_coordinates['s_ra'].values,unit="deg").radian
dec = coordinates.Angle(df_coordinates['s_dec'].values,unit="deg").radian

In [ ]:
# plot
# x arg in rad must be (-2pi,2pi), y arg in rad  must be in ( -pi,pi )
fig = plt.figure(figsize=(16,8))
#ax = fig.add_subplot(111, projection="aitoff")
ax = fig.add_subplot(111, projection="mollweide")
ax.scatter(gp_ra_toplot.radian, gp_radec.dec.radian,c="r",label="Galactic Plane",s=1)
#ax.scatter(ra-np.pi,dec,marker='+',c=all_colors_g)
im = ax.scatter(ra_g.values-np.pi,dec_g.values,marker='+',s=50,lw=3,c=dt,cmap=palette_spectral)
ax.legend()
ax.grid()

cbar = plt.colorbar(im, orientation='horizontal', label='time(days)')
plt.suptitle(instrument + " commissioning 2024")
plt.show()

### Add Deep drilling field

In [ ]:
ddf = {}
ddf["47 Tuc"] = {"name": "47 Tuc Globular Cluster", "ra": 6.02, "dec": -72.08}
ddf["Rubin SV 38 7"] = {"name": "Low Ecliptic Latitude Field", "ra": 37.86, "dec": 6.98}
ddf["Fornax dSph"] = {"name" : "Fornax Dwarf Spheroidal Galaxy", "ra": 40.00, "dec": -34.45}
ddf["ECDFS"] = {"name" : "Extended Chandra Deep Field South", "ra": 53.13, "dec": -28.10}
ddf["EDFS"] = {"name" : "Euclid Deep Field South", "ra": 59.10, "dec": -48.73}
ddf["Rubin SV 95 -25"] = {"name" : "Low Galactic Latitude Field", "ra": 95.00, "dec": -25.00}
ddf["Seagull"] = {"name" : "Seagul Nebular", "ra": 106.23, "dec": -10.51}

In [ ]:
# Extraire les coordonnées RA/DEC depuis le dictionnaire
ddf_ra = []
ddf_dec = []
ddf_labels = []
ddf_names = []

for key, val in ddf.items():
    ddf_ra.append(val["ra"])
    ddf_dec.append(val["dec"])
    ddf_labels.append(key)
    ddf_names.append(val["name"]) 

# Conversion en radians to plot in sky map
ddf_ra_rad = np.radians(ddf_ra) - np.pi  # recentrer RA autour de 0
ddf_dec_rad = np.radians(ddf_dec)

In [ ]:
# plot
# x arg in rad must be (-2pi,2pi), y arg in rad  must be in ( -pi,pi )
fig = plt.figure(figsize=(16,8))
#ax = fig.add_subplot(111, projection="aitoff")
ax = fig.add_subplot(111, projection="mollweide")
ax.scatter(gp_ra_toplot.radian, gp_radec.dec.radian,c="r",label="Galactic Plane",s=1)
#ax.scatter(ra-np.pi,dec,marker='+',c=all_colors_g)
im = ax.scatter(ra_g.values-np.pi,dec_g.values,marker='+',s=50,lw=3,c=dt,cmap=palette_spectral)

#cbar = plt.colorbar(im, orientation='horizontal', label='time(days)')

# Affichage sur la carte
ax.scatter(ddf_ra_rad, ddf_dec_rad, c="k", marker="*", s=200, label="Deep drilling fields")

# Ajouter les noms des champs sur la carte
for ra, dec, label in zip(ddf_ra_rad, ddf_dec_rad, ddf_labels):
    ax.text(ra, dec, label, fontsize=16, fontweight="bold",ha="left", va="bottom", color="k")


# Créer un tableau avec pandas pour les noms
df = pd.DataFrame({
    'Deep Field': ddf_labels,
    'description': ddf_names,
    'RA (deg)': ddf_ra,
    'DEC (deg)': ddf_dec
})

# Créer un tableau sous la carte
table_data = df.values
col_labels = df.columns


# Ajouter un tableau sous la carte avec plus d'espace et ajustement des lignes
table = ax.table(cellText=table_data, colLabels=col_labels, loc='bottom', cellLoc='center', colLoc='center', bbox=[0, -0.2, 1, 0.15])

# Ajuster la taille des lignes (hauteur des cellules)
for key, cell in table.get_celld().items():
    if key[0] == 0:  # En-têtes de colonnes
        cell.set_fontsize(14)
    else:  # Données des lignes
        cell.set_fontsize(12)
    cell.set_height(1.8)  # Ajuste la hauteur des cellules pour éviter qu'elles soient trop écrasées


# Ajouter la colorbar sous le tableau
#cbar = plt.colorbar(im, label='time(days)', ax=ax, pad=0.1, fraction=0.05)
cbar = plt.colorbar(im, label='time(days)', ax=ax)

ax.legend()
ax.grid()
plt.suptitle(instrument + " commissioning 2024",fontsize=16)
plt.subplots_adjust(bottom=0.2)  # Augmenter l'espace en bas pour accueillir la colorbar


## Time dependence

In [ ]:
time = Time(visits['exp_midpt'])

In [ ]:
print('tmin = ',time.min(),' | tmax = ',time.max())

### Airmass vs Time

In [ ]:

# Plot the time vs the specified column
plt.figure(figsize=(12, 4))
plt.plot(time.plot_date, visits['airmass'],'b.')

# Set x-axis to show dates
#plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H'))
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())

# Rotate and format x-axis labels for readability
plt.xticks(rotation=45, ha='right')

plt.xlabel('Time')
plt.ylabel('Airmass')
plt.title(f'Time vs Airmass')
plt.legend()
plt.grid(True)
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()

In [ ]:
# Plot the time vs the specified column
plt.figure(figsize=(12, 4))
plt.plot(time.plot_date, visits['air_temp'],'b.')

# Set x-axis to show dates
#plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H'))
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())

# Rotate and format x-axis labels for readability
plt.xticks(rotation=45, ha='right')

plt.xlabel('Time')
plt.ylabel('air_temp')
plt.title(f'air_temp vs time')
plt.legend()
plt.grid(True)
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()

In [ ]:
# Plot the time vs the specified column
plt.figure(figsize=(12, 4))
plt.plot(time.plot_date, visits['pressure'],'b.')

# Set x-axis to show dates
#plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H'))
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())

# Rotate and format x-axis labels for readability
plt.xticks(rotation=45, ha='right')

plt.xlabel('Time')
plt.ylabel('pressure')
plt.title(f'pressure vs time')
plt.legend()
plt.grid(True)
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()

In [ ]:
# Plot the time vs the specified column
plt.figure(figsize=(12, 4))
plt.plot(time.plot_date, visits['humidity'],'b.')

# Set x-axis to show dates
#plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H'))
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())

# Rotate and format x-axis labels for readability
plt.xticks(rotation=45, ha='right')

plt.xlabel('Time')
plt.ylabel('humidity')
plt.title(f'humidity vs time')
plt.legend()
plt.grid(True)
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()

In [ ]:
# Plot the time vs the specified column
plt.figure(figsize=(12, 4))
plt.plot(time.plot_date, visits['wind_speed'],'b.')

# Set x-axis to show dates
#plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H'))
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())

# Rotate and format x-axis labels for readability
plt.xticks(rotation=45, ha='right')

plt.xlabel('Time')
plt.ylabel('wind_speed')
plt.title(f'wind_speed vs time')
plt.legend()
plt.grid(True)
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()

In [ ]:
# Plot the time vs the specified column
plt.figure(figsize=(12, 4))
plt.plot(time.plot_date, visits['wind_dir'],'b.')

# Set x-axis to show dates
#plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H'))
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())

# Rotate and format x-axis labels for readability
plt.xticks(rotation=45, ha='right')

plt.xlabel('Time')
plt.ylabel('wind_dir')
plt.title(f'wind_dir vs time')
plt.legend()
plt.grid(True)
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()

In [ ]:
# Plot the time vs the specified column
plt.figure(figsize=(12, 4))
plt.plot(time.plot_date, visits['dimm_seeing'],'b.')

# Set x-axis to show dates
#plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H'))
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())

# Rotate and format x-axis labels for readability
plt.xticks(rotation=45, ha='right')

plt.xlabel('Time')
plt.ylabel('dimm_seeing')
plt.title(f'dimm_seeing vs time')
plt.legend()
plt.grid(True)
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()

In [ ]:
# Plot the time vs the specified column
plt.figure(figsize=(12, 4))
plt.plot(time.plot_date, visits['focus_z'],'b.')

# Set x-axis to show dates
#plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H'))
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())

# Rotate and format x-axis labels for readability
plt.xticks(rotation=45, ha='right')

plt.xlabel('Time')
plt.ylabel('focus_z')
plt.title(f'focus_z vs time')
plt.legend()
plt.grid(True)
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()